# Laboratório 1 - Machine Learning

In [ ]:
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', None, 'display.max_columns', None)

## Parte 1 - Pré-processamento

### 1. Carregar dataset

In [ ]:
dataset = pd.read_csv('dataset/02-15-2018.csv')

In [ ]:
dataset[:10]

In [ ]:
colunas_iniciais = dataset.columns
print(f"{len(colunas_iniciais)} colunas")

In [ ]:
colunas_iniciais

In [ ]:
dataset["Label"].value_counts()

### 2. Redimensionamento da coluna Target

In [ ]:
target_classes = dataset["Label"].value_counts().to_dict()
classes = target_classes.keys()

for target in target_classes:
        value = 0 if target == "Benign"else 1
        dataset.replace(to_replace=target, value=value, inplace=True)

In [ ]:
dataset["Label"].value_counts()

### 3. Tratamento de dados ruidosos

In [ ]:
# Deleta a coluna Timestamp
dataset.drop(columns=['Timestamp'], inplace=True)
columns = dataset.columns

In [ ]:
# Substitui valores infinitos por NaN (not a number)
dataset.replace([np.inf, -np.inf], np.nan, inplace=True)

# Deleta todas as linhas que possuem algum valor NaN
dataset.dropna(inplace=True)

### 4. Conversão de tipos incorretos
Verificar ser há alguma coluna com o tipo incorreto, como uma coluna numérica com o tipo object/string

In [ ]:
dataset.dtypes

### 5. Particionamento estratificado dos dados


In [ ]:
from sklearn.model_selection import train_test_split

dataset_train, dataset_test = train_test_split(
        dataset,
        train_size=0.2,
        test_size=0.05, 
        stratify=dataset['Label'], # Manter a proporção de cada tipo nos dois datasets
        random_state=42 # Escolha aleatória que pode ser reproduzida posteriormente
    )

In [ ]:
print(f"Dataset de treino: {len(dataset_train)} linhas")
print(f"Dataset de teste: {len(dataset_test)} linhas")

### 6. Demais etapas
Existem **outras etapas de pré-processamento** que devem ser aplicadas ao dataset, no entanto, **por motivos de tempo** iremos utilizar algumas funções de pré-processamento embutidas no próprio autoML **Pycaret**

Nesse sentido, as seguintes etapas serão aplicadas pelo próprio **Pycaret**:
- Normalização
- Remoção de outlieers
- Seleção de colunas

## Parte 2 - Treinamento

In [ ]:
from pycaret.classification import setup, compare_models, pull, save_model, plot_model, predict_model
import pandas as pd


# Configuração do ambiente do PyCaret
clf = setup(
    data=dataset_train,
    test_data=dataset_test,
    target='Label',
    
    # --- Pré-processamento desejado ---
    normalize=True,              # ativa normalização
    normalize_method='minmax',   # método de normalização min-max
    
    remove_outliers=False,        # Desativa remove_outliers por causa do grande consumo de RAM
    # remove_outliers=True,        # remove outliers
    # outliers_method='iforest',   # usa Isolation Forest
    
    pca=False,                    # Desativa PCA por causa do grande consumo de RAM
    # pca=True,                    # ativa PCA
    # pca_method='linear',         # PCA linear
    # pca_components=0.95,         # mantém 95% da variância (ajuste se quiser)
    
    # --- Desativar transformações automáticas ---
    imputation_type=None,            # não faz imputação
    categorical_features=None,       # não faz encoding
    polynomial_features=False,
    low_variance_threshold=None,
    remove_multicollinearity=False,
    feature_selection=False,
    transformation=False,

    # --- Outras configurações ---
    session_id=123,              # reprodutibilidade
    use_gpu=False,                # acelera se houver GPU disponível
    html=False                   # evita renderização interativa
)


In [ ]:
['dt', ]

In [ ]:
# Treina e compara modelos automaticamente
best_model = compare_models(include=['knn', 'dt','rf', 'lr', 'mlp'])

In [ ]:
# Visualiza o ranking de modelos
results = pull()
results.to_csv('resultados_modelos.csv', index=False)
print(results)

In [ ]:
results.to_pickle('output/resultados_modelos.pkl')


In [ ]:
save_model(best_model, 'output/melhor_modelo')


## Parte 3 - Avaliação

In [ ]:
plot_model(best_model, plot='confusion_matrix')


## Parte 4 - Classificação

In [ ]:
# 🔹 Usa o melhor modelo encontrado para prever todo o dataset
predictions = predict_model(best_model, data=dataset)

In [ ]:
# from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# # Gera a matriz
# cm = confusion_matrix(predictions['Label'], predictions['Label_predicted'])

# # Exibe a matriz com rótulos
# disp = ConfusionMatrixDisplay(confusion_matrix=cm)
# disp.plot(cmap='Blues')
